In [1]:
# Importing modules
import pandas as pd
import os

# Load the regular expression library
import re

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk as nltk
nltk.download('stopwords')
nltk.download('wordnet')
import gensim
from gensim.utils import simple_preprocess
from gensim.models.wrappers import LdaMallet
from nltk.corpus import stopwords
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\megha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\megha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
diabetes_class = pd.read_csv('diabetes_2020_classification_train.csv')

In [3]:
diabetes_class

,Post ID,Author,Publish Date,Body,Dominant,month,classfication,permalink,Title
0,fkg6fbo,timeflieswhen,2020-03-13 18:58,Did your blood sugar go down finally?,Topic 7,20-Mar,noncovid,/r/diabetes/comments/ey8ebn/high_blood_sugar_when_fasting/fkg6fbo/,0
1,fqej4ew,GjjWhiteBelt,2020-05-12 11:33,So eat right and exercise hard. Type two is your fault. Get rid of it.,Topic 8,20-May,noncovid,/r/diabetes/comments/f212mt/just_diagnosed/fqej4ew/,0
2,fkkmuzs,Frjttr,2020-03-15 8:53,"It is indeed, but better indulge partying than having a non-life ðŸ¤·ðŸ»â€â™‚ï¸ And even once...",Topic 10,20-Mar,noncovid,/r/diabetes/comments/f33pw5/why_why_why_do_i_do_this/fkkmuzs/,0
3,fj2il08,OneFuzzyPillow,2020-02-28 18:49,"For blood sugar, it doesn't seem to affect it. But! You want to stay hydrated for sure, so mak...",Topic 8,20-Feb,noncovid,/r/diabetes/comments/f9lhz0/is_drinking_4_diet_cokes_a_day_bad_for_you/fj2il08/,0
4,fj00v3d,Route22,2020-02-28 1:56,"An interesting observation... you might need to try another one. You know, for science.",Topic 5,20-Feb,noncovid,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj00v3d/,0
...,...,...,...,...,...,...,...,...,...
4613,fjhku6k,TeaAndCrackers,2020-03-04 11:27,"I have a list of foods that don't raise my blood sugar, if you'd like to see it. I eat less than...",Topic 7,20-Mar,noncovid,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjhku6k/,0
4614,fjim8u4,dezilove85,2020-03-04 17:12,Would love the list,Topic 5,20-Mar,noncovid,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjim8u4/,0
4615,fjinen3,TeaAndCrackers,2020-03-04 17:24,"ANIMAL PROTEIN:\nBeef, pork, poultry, lamb, game animal ( all parts - no fillers) 85 - 100% fat....",Topic 1,20-Mar,noncovid,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjinen3/,0
4616,fjinyy3,dezilove85,2020-03-04 17:30,Thank u,Topic 1,20-Mar,noncovid,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjinyy3/,0


In [4]:
#remove any blank texts
diabetes_class = diabetes_class[diabetes_class['Body'].notna()]
diabetes_class

,Post ID,Author,Publish Date,Body,Dominant,month,classfication,permalink,Title
0,fkg6fbo,timeflieswhen,2020-03-13 18:58,Did your blood sugar go down finally?,Topic 7,20-Mar,noncovid,/r/diabetes/comments/ey8ebn/high_blood_sugar_when_fasting/fkg6fbo/,0
1,fqej4ew,GjjWhiteBelt,2020-05-12 11:33,So eat right and exercise hard. Type two is your fault. Get rid of it.,Topic 8,20-May,noncovid,/r/diabetes/comments/f212mt/just_diagnosed/fqej4ew/,0
2,fkkmuzs,Frjttr,2020-03-15 8:53,"It is indeed, but better indulge partying than having a non-life ðŸ¤·ðŸ»â€â™‚ï¸ And even once...",Topic 10,20-Mar,noncovid,/r/diabetes/comments/f33pw5/why_why_why_do_i_do_this/fkkmuzs/,0
3,fj2il08,OneFuzzyPillow,2020-02-28 18:49,"For blood sugar, it doesn't seem to affect it. But! You want to stay hydrated for sure, so mak...",Topic 8,20-Feb,noncovid,/r/diabetes/comments/f9lhz0/is_drinking_4_diet_cokes_a_day_bad_for_you/fj2il08/,0
4,fj00v3d,Route22,2020-02-28 1:56,"An interesting observation... you might need to try another one. You know, for science.",Topic 5,20-Feb,noncovid,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj00v3d/,0
...,...,...,...,...,...,...,...,...,...
4613,fjhku6k,TeaAndCrackers,2020-03-04 11:27,"I have a list of foods that don't raise my blood sugar, if you'd like to see it. I eat less than...",Topic 7,20-Mar,noncovid,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjhku6k/,0
4614,fjim8u4,dezilove85,2020-03-04 17:12,Would love the list,Topic 5,20-Mar,noncovid,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjim8u4/,0
4615,fjinen3,TeaAndCrackers,2020-03-04 17:24,"ANIMAL PROTEIN:\nBeef, pork, poultry, lamb, game animal ( all parts - no fillers) 85 - 100% fat....",Topic 1,20-Mar,noncovid,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjinen3/,0
4616,fjinyy3,dezilove85,2020-03-04 17:30,Thank u,Topic 1,20-Mar,noncovid,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjinyy3/,0


In [5]:
#stopword libraries https://stackabuse.com/removing-stop-words-from-strings-in-python/
#gensim stopwords
from gensim.parsing.preprocessing import remove_stopwords


#text = "Nick likes to play football, however he is not too fond of tennis."
#filtered_sentence = remove_stopwords(text)

#print(filtered_sentence)
#def remove_stopwords(text):
    #text = [word for word in text if word not in stopword]
    #print(text)
    #return text
    
#diabetes2018['text_nonstop'] = diabetes2018['text_tokenized'].apply(lambda x: remove_stopwords(x))
#diabetes2018.head(10)

diabetes_class['text_processed'] = diabetes_class['Body'].apply(lambda x: remove_stopwords(x))
diabetes_class

<ipython-input-5-869c6e621ecd>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diabetes_class['text_processed'] = diabetes_class['Body'].apply(lambda x: remove_stopwords(x))


,Post ID,Author,Publish Date,Body,Dominant,month,classfication,permalink,Title,text_processed
0,fkg6fbo,timeflieswhen,2020-03-13 18:58,Did your blood sugar go down finally?,Topic 7,20-Mar,noncovid,/r/diabetes/comments/ey8ebn/high_blood_sugar_when_fasting/fkg6fbo/,0,Did blood sugar finally?
1,fqej4ew,GjjWhiteBelt,2020-05-12 11:33,So eat right and exercise hard. Type two is your fault. Get rid of it.,Topic 8,20-May,noncovid,/r/diabetes/comments/f212mt/just_diagnosed/fqej4ew/,0,So eat right exercise hard. Type fault. Get rid it.
2,fkkmuzs,Frjttr,2020-03-15 8:53,"It is indeed, but better indulge partying than having a non-life ðŸ¤·ðŸ»â€â™‚ï¸ And even once...",Topic 10,20-Mar,noncovid,/r/diabetes/comments/f33pw5/why_why_why_do_i_do_this/fkkmuzs/,0,"It indeed, better indulge partying having non-life ðŸ¤·ðŸ»â€â™‚ï¸ And year dangerous, wonâ€™t..."
3,fj2il08,OneFuzzyPillow,2020-02-28 18:49,"For blood sugar, it doesn't seem to affect it. But! You want to stay hydrated for sure, so mak...",Topic 8,20-Feb,noncovid,/r/diabetes/comments/f9lhz0/is_drinking_4_diet_cokes_a_day_bad_for_you/fj2il08/,0,"For blood sugar, doesn't affect it. But! You want stay hydrated sure, sure drink water too! I dr..."
4,fj00v3d,Route22,2020-02-28 1:56,"An interesting observation... you might need to try another one. You know, for science.",Topic 5,20-Feb,noncovid,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj00v3d/,0,"An interesting observation... need try one. You know, science."
...,...,...,...,...,...,...,...,...,...,...
4613,fjhku6k,TeaAndCrackers,2020-03-04 11:27,"I have a list of foods that don't raise my blood sugar, if you'd like to see it. I eat less than...",Topic 7,20-Mar,noncovid,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjhku6k/,0,"I list foods don't raise blood sugar, you'd like it. I eat 50 carbs day A1c 5s. Eat meter: Check..."
4614,fjim8u4,dezilove85,2020-03-04 17:12,Would love the list,Topic 5,20-Mar,noncovid,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjim8u4/,0,Would love list
4615,fjinen3,TeaAndCrackers,2020-03-04 17:24,"ANIMAL PROTEIN:\nBeef, pork, poultry, lamb, game animal ( all parts - no fillers) 85 - 100% fat....",Topic 1,20-Mar,noncovid,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjinen3/,0,"ANIMAL PROTEIN: Beef, pork, poultry, lamb, game animal ( parts - fillers) 85 - 100% fat. Cured m..."
4616,fjinyy3,dezilove85,2020-03-04 17:30,Thank u,Topic 1,20-Mar,noncovid,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjinyy3/,0,Thank u


In [6]:
#move column
diabetes_class = diabetes_class[['Post ID', 'Author', 'Publish Date', 'Body', 'text_processed', 'classfication', 'Dominant', 'month', 'permalink', 'Title']]
diabetes_class

,Post ID,Author,Publish Date,Body,text_processed,classfication,Dominant,month,permalink,Title
0,fkg6fbo,timeflieswhen,2020-03-13 18:58,Did your blood sugar go down finally?,Did blood sugar finally?,noncovid,Topic 7,20-Mar,/r/diabetes/comments/ey8ebn/high_blood_sugar_when_fasting/fkg6fbo/,0
1,fqej4ew,GjjWhiteBelt,2020-05-12 11:33,So eat right and exercise hard. Type two is your fault. Get rid of it.,So eat right exercise hard. Type fault. Get rid it.,noncovid,Topic 8,20-May,/r/diabetes/comments/f212mt/just_diagnosed/fqej4ew/,0
2,fkkmuzs,Frjttr,2020-03-15 8:53,"It is indeed, but better indulge partying than having a non-life ðŸ¤·ðŸ»â€â™‚ï¸ And even once...","It indeed, better indulge partying having non-life ðŸ¤·ðŸ»â€â™‚ï¸ And year dangerous, wonâ€™t...",noncovid,Topic 10,20-Mar,/r/diabetes/comments/f33pw5/why_why_why_do_i_do_this/fkkmuzs/,0
3,fj2il08,OneFuzzyPillow,2020-02-28 18:49,"For blood sugar, it doesn't seem to affect it. But! You want to stay hydrated for sure, so mak...","For blood sugar, doesn't affect it. But! You want stay hydrated sure, sure drink water too! I dr...",noncovid,Topic 8,20-Feb,/r/diabetes/comments/f9lhz0/is_drinking_4_diet_cokes_a_day_bad_for_you/fj2il08/,0
4,fj00v3d,Route22,2020-02-28 1:56,"An interesting observation... you might need to try another one. You know, for science.","An interesting observation... need try one. You know, science.",noncovid,Topic 5,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj00v3d/,0
...,...,...,...,...,...,...,...,...,...,...
4613,fjhku6k,TeaAndCrackers,2020-03-04 11:27,"I have a list of foods that don't raise my blood sugar, if you'd like to see it. I eat less than...","I list foods don't raise blood sugar, you'd like it. I eat 50 carbs day A1c 5s. Eat meter: Check...",noncovid,Topic 7,20-Mar,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjhku6k/,0
4614,fjim8u4,dezilove85,2020-03-04 17:12,Would love the list,Would love list,noncovid,Topic 5,20-Mar,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjim8u4/,0
4615,fjinen3,TeaAndCrackers,2020-03-04 17:24,"ANIMAL PROTEIN:\nBeef, pork, poultry, lamb, game animal ( all parts - no fillers) 85 - 100% fat....","ANIMAL PROTEIN: Beef, pork, poultry, lamb, game animal ( parts - fillers) 85 - 100% fat. Cured m...",noncovid,Topic 1,20-Mar,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjinen3/,0
4616,fjinyy3,dezilove85,2020-03-04 17:30,Thank u,Thank u,noncovid,Topic 1,20-Mar,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjinyy3/,0


In [7]:
#switch to lower case
diabetes_class['text_processed'] = \
diabetes_class['text_processed'].map(lambda x: x.lower())
diabetes_class

,Post ID,Author,Publish Date,Body,text_processed,classfication,Dominant,month,permalink,Title
0,fkg6fbo,timeflieswhen,2020-03-13 18:58,Did your blood sugar go down finally?,did blood sugar finally?,noncovid,Topic 7,20-Mar,/r/diabetes/comments/ey8ebn/high_blood_sugar_when_fasting/fkg6fbo/,0
1,fqej4ew,GjjWhiteBelt,2020-05-12 11:33,So eat right and exercise hard. Type two is your fault. Get rid of it.,so eat right exercise hard. type fault. get rid it.,noncovid,Topic 8,20-May,/r/diabetes/comments/f212mt/just_diagnosed/fqej4ew/,0
2,fkkmuzs,Frjttr,2020-03-15 8:53,"It is indeed, but better indulge partying than having a non-life ðŸ¤·ðŸ»â€â™‚ï¸ And even once...","it indeed, better indulge partying having non-life ðÿ¤·ðÿ»â€â™‚ï¸ and year dangerous, wonâ€™t...",noncovid,Topic 10,20-Mar,/r/diabetes/comments/f33pw5/why_why_why_do_i_do_this/fkkmuzs/,0
3,fj2il08,OneFuzzyPillow,2020-02-28 18:49,"For blood sugar, it doesn't seem to affect it. But! You want to stay hydrated for sure, so mak...","for blood sugar, doesn't affect it. but! you want stay hydrated sure, sure drink water too! i dr...",noncovid,Topic 8,20-Feb,/r/diabetes/comments/f9lhz0/is_drinking_4_diet_cokes_a_day_bad_for_you/fj2il08/,0
4,fj00v3d,Route22,2020-02-28 1:56,"An interesting observation... you might need to try another one. You know, for science.","an interesting observation... need try one. you know, science.",noncovid,Topic 5,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj00v3d/,0
...,...,...,...,...,...,...,...,...,...,...
4613,fjhku6k,TeaAndCrackers,2020-03-04 11:27,"I have a list of foods that don't raise my blood sugar, if you'd like to see it. I eat less than...","i list foods don't raise blood sugar, you'd like it. i eat 50 carbs day a1c 5s. eat meter: check...",noncovid,Topic 7,20-Mar,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjhku6k/,0
4614,fjim8u4,dezilove85,2020-03-04 17:12,Would love the list,would love list,noncovid,Topic 5,20-Mar,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjim8u4/,0
4615,fjinen3,TeaAndCrackers,2020-03-04 17:24,"ANIMAL PROTEIN:\nBeef, pork, poultry, lamb, game animal ( all parts - no fillers) 85 - 100% fat....","animal protein: beef, pork, poultry, lamb, game animal ( parts - fillers) 85 - 100% fat. cured m...",noncovid,Topic 1,20-Mar,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjinen3/,0
4616,fjinyy3,dezilove85,2020-03-04 17:30,Thank u,thank u,noncovid,Topic 1,20-Mar,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjinyy3/,0


In [8]:
#Remove punctuations & upper case
#Remove stopwords
#Lammetization/stemming - Tranforming any form of a word to its root word
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [9]:
#remove puncutation
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

diabetes_class['text_processed'] = diabetes_class['text_processed'].apply(lambda x: remove_punct(x))
diabetes_class.head(10)

,Post ID,Author,Publish Date,Body,text_processed,classfication,Dominant,month,permalink,Title
0,fkg6fbo,timeflieswhen,2020-03-13 18:58,Did your blood sugar go down finally?,did blood sugar finally,noncovid,Topic 7,20-Mar,/r/diabetes/comments/ey8ebn/high_blood_sugar_when_fasting/fkg6fbo/,0
1,fqej4ew,GjjWhiteBelt,2020-05-12 11:33,So eat right and exercise hard. Type two is your fault. Get rid of it.,so eat right exercise hard type fault get rid it,noncovid,Topic 8,20-May,/r/diabetes/comments/f212mt/just_diagnosed/fqej4ew/,0
2,fkkmuzs,Frjttr,2020-03-15 8:53,"It is indeed, but better indulge partying than having a non-life ðŸ¤·ðŸ»â€â™‚ï¸ And even once...",it indeed better indulge partying having nonlife ðÿ¤·ðÿ»â€â™‚ï¸ and year dangerous wonâ€™t he...,noncovid,Topic 10,20-Mar,/r/diabetes/comments/f33pw5/why_why_why_do_i_do_this/fkkmuzs/,0
3,fj2il08,OneFuzzyPillow,2020-02-28 18:49,"For blood sugar, it doesn't seem to affect it. But! You want to stay hydrated for sure, so mak...",for blood sugar doesnt affect it but you want stay hydrated sure sure drink water too i drink lo...,noncovid,Topic 8,20-Feb,/r/diabetes/comments/f9lhz0/is_drinking_4_diet_cokes_a_day_bad_for_you/fj2il08/,0
4,fj00v3d,Route22,2020-02-28 1:56,"An interesting observation... you might need to try another one. You know, for science.",an interesting observation need try one you know science,noncovid,Topic 5,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj00v3d/,0
5,fj033gg,Person_Something,2020-02-28 2:47,"I donâ€™t really have a problem with ice cream. I eat it quite often, and my bg stays stable. Ma...",i donâ€™t problem ice cream i eat often bg stays stable maybe itâ€™s personal reaction i drink m...,noncovid,Topic 1,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj033gg/,0
6,fj08cy7,goddessabove,2020-02-28 4:28,But was it worth it? The shamrock shake was my favorite and now they added Oreos. :( I thought a...,but worth it the shamrock shake favorite added oreos i thought getting taking bites i dont know...,noncovid,Topic 8,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj08cy7/,0
7,fj092j6,Grimnex2507,2020-02-28 4:39,I love you for that joke &lt;3,i love joke lt,noncovid,Topic 4,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj092j6/,0
8,fj0qzir,TheSessionMan,2020-02-28 8:05,You can pre-bolus with MDI too.,you prebolus mdi too,noncovid,Topic 3,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj0qzir/,0
9,fj0zmzv,investinlove,2020-02-28 9:19,"My best philosophy is to deny myself nothing, but only take a single bite of the offending food....",my best philosophy deny nothing single bite offending food eating entire dessert occur me,noncovid,Topic 1,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj0zmzv/,0


In [10]:
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
diabetes_class['text_processed'] = diabetes_class['text_processed'].astype(str)
diabetes_class['text_processed']= [word_tokenize(entry) for entry in diabetes_class['text_processed']]
diabetes_class

,Post ID,Author,Publish Date,Body,text_processed,classfication,Dominant,month,permalink,Title
0,fkg6fbo,timeflieswhen,2020-03-13 18:58,Did your blood sugar go down finally?,"[did, blood, sugar, finally]",noncovid,Topic 7,20-Mar,/r/diabetes/comments/ey8ebn/high_blood_sugar_when_fasting/fkg6fbo/,0
1,fqej4ew,GjjWhiteBelt,2020-05-12 11:33,So eat right and exercise hard. Type two is your fault. Get rid of it.,"[so, eat, right, exercise, hard, type, fault, get, rid, it]",noncovid,Topic 8,20-May,/r/diabetes/comments/f212mt/just_diagnosed/fqej4ew/,0
2,fkkmuzs,Frjttr,2020-03-15 8:53,"It is indeed, but better indulge partying than having a non-life ðŸ¤·ðŸ»â€â™‚ï¸ And even once...","[it, indeed, better, indulge, partying, having, nonlife, ðÿ¤·ðÿ, », â€â™‚ï¸, and, year, dange...",noncovid,Topic 10,20-Mar,/r/diabetes/comments/f33pw5/why_why_why_do_i_do_this/fkkmuzs/,0
3,fj2il08,OneFuzzyPillow,2020-02-28 18:49,"For blood sugar, it doesn't seem to affect it. But! You want to stay hydrated for sure, so mak...","[for, blood, sugar, doesnt, affect, it, but, you, want, stay, hydrated, sure, sure, drink, water...",noncovid,Topic 8,20-Feb,/r/diabetes/comments/f9lhz0/is_drinking_4_diet_cokes_a_day_bad_for_you/fj2il08/,0
4,fj00v3d,Route22,2020-02-28 1:56,"An interesting observation... you might need to try another one. You know, for science.","[an, interesting, observation, need, try, one, you, know, science]",noncovid,Topic 5,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj00v3d/,0
...,...,...,...,...,...,...,...,...,...,...
4613,fjhku6k,TeaAndCrackers,2020-03-04 11:27,"I have a list of foods that don't raise my blood sugar, if you'd like to see it. I eat less than...","[i, list, foods, dont, raise, blood, sugar, youd, like, it, i, eat, carbs, day, ac, s, eat, mete...",noncovid,Topic 7,20-Mar,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjhku6k/,0
4614,fjim8u4,dezilove85,2020-03-04 17:12,Would love the list,"[would, love, list]",noncovid,Topic 5,20-Mar,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjim8u4/,0
4615,fjinen3,TeaAndCrackers,2020-03-04 17:24,"ANIMAL PROTEIN:\nBeef, pork, poultry, lamb, game animal ( all parts - no fillers) 85 - 100% fat....","[animal, protein, beef, pork, poultry, lamb, game, animal, parts, fillers, fat, cured, meats, am...",noncovid,Topic 1,20-Mar,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjinen3/,0
4616,fjinyy3,dezilove85,2020-03-04 17:30,Thank u,"[thank, u]",noncovid,Topic 1,20-Mar,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjinyy3/,0


In [11]:
#remove nlk stopwords
stopword = nltk.corpus.stopwords.words('english')
stopword.extend(['get', "didnt", 'would', 'a', 'hey', 'hello', 'hi', "ive", "im", 'like', 'also',"don't", "I've", "I'm", "I'll", 'id', 'dont', 'wont', 'isnt', 'lol', 'youre', 'youll', 'doesnt'])
stopword

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [12]:
#remove nlk stopwords
def remove_nlk_stopwords(text):
    text = [word for word in text if word not in stopword]
    print(text)
    return text
    
diabetes_class['text_processed'] = diabetes_class['text_processed'].apply(lambda x: remove_nlk_stopwords(x))
diabetes_class.head(10)

['blood', 'sugar', 'finally']
['eat', 'right', 'exercise', 'hard', 'type', 'fault', 'rid']
['indeed', 'better', 'indulge', 'partying', 'nonlife', 'ðÿ¤·ðÿ\x8f', '»', 'â€\x8dâ™‚ï¸\x8f', 'year', 'dangerous', 'wonâ€™t', 'heal', 'damage', 'caused']
['blood', 'sugar', 'affect', 'want', 'stay', 'hydrated', 'sure', 'sure', 'drink', 'water', 'drink', 'lot', 'seltzer', 'water', 'got', 'fizzies', 'flavor', 'sugar', 'counts', 'daily', 'water', 'intake']
['interesting', 'observation', 'need', 'try', 'one', 'know', 'science']
['donâ€™t', 'problem', 'ice', 'cream', 'eat', 'often', 'bg', 'stays', 'stable', 'maybe', 'itâ€™s', 'personal', 'reaction', 'drink', 'milk', 'inject', 'hiiiiiiiigh']
['worth', 'shamrock', 'shake', 'favorite', 'added', 'oreos', 'thought', 'getting', 'taking', 'bites', 'know', 'bother']
['love', 'joke', 'lt']
['prebolus', 'mdi']
['best', 'philosophy', 'deny', 'nothing', 'single', 'bite', 'offending', 'food', 'eating', 'entire', 'dessert', 'occur']
['itâ€™s', 'basically', 'mint', '

['disclose', 'weeks', 'pregnant', 'normally', 'type', 'diabetic', 'ac', 'becuase', 'pregnancy', 'sugars', 'need', 'controlled', 'told', 'insulin', 'bedtime', 'kinda', 'worked', 'inject', 'usually', 'mins', 'bed', 'eat', 'fasting', 'number', 'morning', 'according', 'endocrinologist', 'breakfast', 'high', 'lunch', 'time', 'dinner', 'normal', 'normal']
['welcome', 'pregnancy', 'diabetes', 'insulin', 'needs', 'increase', 'pregnancy', 'first', 'fasting', 'blood', 'sugar', 'increases', 'eventually', 'youâ€™ll', 'need', 'insulin', 'cover', 'meals', 'sure', 'seeing', 'endocrinologist', 'meter', 'maternal', 'fetal', 'medicine', 'specialist']
['spammers', 'suck']
['grandmother', 'diabetes', 'type', 'suffers', 'lot', 'pain', 'toes', 'feet', 'tried', 'rubbing', 'putting', 'ointment', 'feet', 'gets', 'great', 'deal', 'pain', 'takes', 'medicine', 'meds', 'needs', 'new', 'issue', 'affects', 'give', 'whos', 'advice', 'help']
['sound', 'weird', 'nerve', 'painphantom', 'pain', 'large', 'injury', 'destro

['everyone', 'new', 'community', 'diagnosed', 'type', 'sugars', 'constantly', 'kind', 'erratic', 'ac', 'year', 'ago', 'months', 'ago', 'trying', 'work', 'quarantine', 'kind', 'depressed', 'trying', 'walk', 'dog', 'day', 'activity', 'know', 'know', 'excuses', 'struggle', 'food', 'addiction', 'know', 'however', 'sugars', 'high', 'feel', 'effects', 'wake', 'hours', 'pee', 'thirsty', 'desert', 'ketone', 'urine', 'strip', 'tested', 'high', 'want', 'hospital', 'symptoms', 'ketoacidosis', 'thirst', 'nausea', 'confusion', 'abdominal', 'pain', 'etc', 'vitamins', 'daily', 'wondering', 'vitamins', 'ketones', 'high', 'want', 'worse', 'drinking', 'water', 'oz', 'far', 'today', 'best', 'fast', 'couple', 'days', 'appreciate', 'help', 'advice']
['blood', 'sugar', 'currently']
[]
['whenever', 'keystones', 'high', 'school', 'nurse', 'recommended', 'drink', 'water', 'rest', 'helped']
['ketones', 'reading', 'high', 'insulin', 'need', 'insulin', 'straight', 'away', 'need', 'visit', 'doctor', 'hospital', 'a

['wouldnt', 'going', 'case', 'need', 'validation', 'even', 'reason', 'lower', 'risk', 'catching', 'higher', 'risk', 'dying', 'complications', 'worth', 'thatno', 'one', 'going', 'confirmed', 'cases', 'office', 'shutting', 'cleaning', 'quarantining', 'office', 'probably', 'weeks']
['fuck', 'yeah', 'bring', 'overtime', 'day', 'come']
['speculations', 'facts', 'different', 'things', 'sure', 'point', 'helpful', 'way', 'present', 'makes', 'sound', 'safer', 'nondiabetics', 'thats', 'pretty', 'damn', 'dangerous', 'thing', 'imply']
['deleted', 'avoid', 'conflict']
['use', 'loop', 'application', 'iphone', 'open', 'loop', 'mode', 'pulls', 'dexcom', 'readings', 'real', 'time', 'places', 'healthkit', 'inpen', 'finds']
['type', 'carb', 'white', 'brown', 'matterall', 'carbs', 'cause', 'spikes', 'case', 'ts', 'recommend', 'rketo', 'cutting', 'carbs', 'likely', 'improvement', 'alone', 'fasting', 'good', 'regularly', 'movingeven', 'walking', 'need', 'lift', 'arnold', 'risk', 'health', 'gym', 'pandemic',

['ahh', 'maybe', 'havenâ€™t', 'cheddar', 'ones', 'yet']
['agreed', 'grease', 'salt', 'whisps', 'fan']
['united', 'states', 'trader', 'joeâ€™s', 'tasty', 'cheese', 'chip', 'much', 'better', 'moon', 'cheese', 'ðÿ§€']
['tjâ€™a', 'cheese', 'crisps', 'good', 'bbq', 'whisps']
['super', 'easy']
['agree', 'cheddar', 'gouda', 'bad', 'parmesan', 'great']
['cant', 'whisps', 'either', 'something', 'consistency', 'drives', 'crazy']
['moon', 'cheese', 'parmesan', 'whips', 'good', 'southern', 'recipe', 'small', 'batch', 'pork', 'rinds', 'better', 'typical', 'pork', 'rinds', 'pineapple', 'anchor', 'chile', 'flavor', 'excellent', 'light', 'fluffy']
['recipe', 'link']
['cheese', 'crisps', 'easy', 'microwave']
['microwave']
['taste', 'super', 'bad', 'texture', 'weird', 'wouldnâ€™t', 'recommend', 'definitely', 'try', 'know']
['definitely', 'try', 'love', 'snacking', 'need', 'stuff', 'snack']
['whats', 'serving', 'size']
['serving', 'size', 'pieces', 'itâ€™s', 'g', 'carb', 'serving', 'size', 'bag']
['iâ€™v

['net', 'carbs', 'game', 'dibetics', 'need', 'know', 'overall', 'carb', 'count', 'every', 'doctor', 'nutritionist', 'said', 'net', 'carb', 'game', 'misleading', 'incorrect']
['sugar', 'alcohols', 'minimal', 'effect', 'blood', 'sugar', 'considered', 'keto', 'safe', 'erythritol', 'xylitol', 'though', 'donâ€™t', 'xylitol', 'house', 'itâ€™s', 'poisonous', 'dogs', 'safely', 'deduct', 'carbohydrates', 'sugar', 'alcohols', 'sugar', 'alcohols', 'maltitol', 'affect', 'blood', 'sugar', 'deduct', 'carbohydrates', 'avoid', 'maltitol', 'itâ€™s', 'causes', 'extreme', 'gastric', 'distress', 'xylitol', 'gives', 'upset', 'stomach', 'consume', 'erythritol', 'generally', 'havenâ€™t', 'long', 'time']
['fiber', 'digestible', 'negligible', 'affect', 'blood', 'sugar', 'safety', 'deduct', 'overall', 'carbohydrate', 'content', 'fact', 'consuming', 'high', 'fiber', 'foods', 'overall', 'positive', 'impact', 'blood', 'sugar', 'slow', 'absorption', 'watch', 'foods', 'list', 'â€œmodified', 'starchesâ€\x9d', 'fiber'

['vodka', 'makes', 'spike', 'way', 'else', 'vodka', 'sugar', 'free', 'redbull']
['drink', 'vacation', 'twice', 'year', 'nights', 'vacation', 'drink', 'drink', 'lot', 'itâ€™s', 'vodka', 'seltzer', 'iâ€™ll', 'drink', 'red', 'wine', 'moonshine', 'hangovers', 'little', 'worse', 'thatâ€™s', 'difference']
['depends', 'liquor', 'affects', 'learned', 'doctors', 'alcohol', 'lower', 'someoneâ€™s', 'blood', 'sugar', 'especially', 'time', 'metformin', 'taken', 'understood', 'heard', 'raised', 'blood', 'sugar', 'sugar', 'carbs', 'drank', 'glass', 'wine', 'weeks', 'ago', 'dinner', 'didnâ€™t', 'think', 'took', 'mg', 'metformin', 'never', 'drink', 'felt', 'strange', 'checked', 'glucose', 'hour', 'heavily', 'carbed', 'meal', 'nobody', 'tell', 'drink', 'affect', 'differently', 'else', 'test', 'works', 'careful']
['honestly', 'got', 'wait', 'come', 'time', 'ozempic']
['tried', 'force', 'try', 'â€œstay', 'normalâ€\x9d', 'activities', 'eat', 'tiny', 'snacks', 'day', 'takes', 'mind', 'cant', 'figure', 'what

['please', 'mind', 'newly', 'diagnosed', 'person', 'learning', 'way', 'tried', 'sample', 'week', 'libre', 'experiment', 'different', 'foods', 'impacted', 'blood', 'sugar', 'levels', 'surprising', 'results', 'great', 'outcome', 'two', 'weeks', 'ago', 'given', 'sample', 'sensor', 'finding', 'phone', 'wasnt', 'compatible', 'ordered', 'sensor', 'device', 'works', 'ended', 'checking', 'blood', 'sugar', 'times', 'day', 'average', 'turns', 'enemy', 'wheat', 'greatest', 'enemy', 'wheat', 'contains', 'yeast', 'holy', 'smokes', 'went', 'eating', 'piece', 'bread', 'weirdest', 'thing', 'finding', 'surprising', 'foods', 'lowered', 'blood', 'sugar', 'levels', 'eating', 'egg', 'noodles', 'durum', 'wheat', 'chicken', 'livers', 'lowered', 'levels', 'eating', 'chicken', 'livers', 'reduced', 'levels', 'long', 'time', 'favorite', 'experiment', 'saved', 'today', 'big', 'mac', 'combo', 'fries', 'diet', 'coke', 'went', 'thing', 'stayed', 'level', 'hours', 'tried', 'small', 'home', 'blueberry', 'muffin', 'bre

['got', 'prescribed', 'invokana', 'mg', 'tablet', 'day', 'meal', 'ac', 'diabetes', 'control', 'managed', 'drop', 'ac', 'think', 'lost', 'way', 'good', 'eating', 'working', 'gained', 'weight', 'currently', 'working', 'getting', 'commercial', 'pilot', 'license', 'invokana', 'fly', 'list', 'faa', 'wondering', 'lower', 'ac', 'time', 'check', 'invokana', 'currently', 'eating', 'calories', 'day', 'working', 'times', 'week', 'hour', 'hour', 'half', 'depending', 'work']
['yesterday', 'bowl', 'shells', 'cheese', 'today', 'pieces', 'crust', 'veggie', 'pizza', 'feel', 'guilty', 'itâ€™s', 'easy']
['feel', 'bad', 'long', 'got', 'track', 'thats', 'matters']
['youve', 'rough', 'couple', 'days', 'tomorrow', 'better', 'remember', 'theres', 'place', 'diet', 'junk', 'food', 'moderation', 'pieces', 'pizza', 'day', 'excessive', 'piece', 'pizza', 'ok']
['usually', 'piece', 'pizza', 'usually', 'inches', 'diameter', 'though']
['gtyesterday', 'bowl', 'shells', 'cheese', 'today', 'pieces', 'crust', 'veggie', 'p

,Post ID,Author,Publish Date,Body,text_processed,classfication,Dominant,month,permalink,Title
0,fkg6fbo,timeflieswhen,2020-03-13 18:58,Did your blood sugar go down finally?,"[blood, sugar, finally]",noncovid,Topic 7,20-Mar,/r/diabetes/comments/ey8ebn/high_blood_sugar_when_fasting/fkg6fbo/,0
1,fqej4ew,GjjWhiteBelt,2020-05-12 11:33,So eat right and exercise hard. Type two is your fault. Get rid of it.,"[eat, right, exercise, hard, type, fault, rid]",noncovid,Topic 8,20-May,/r/diabetes/comments/f212mt/just_diagnosed/fqej4ew/,0
2,fkkmuzs,Frjttr,2020-03-15 8:53,"It is indeed, but better indulge partying than having a non-life ðŸ¤·ðŸ»â€â™‚ï¸ And even once...","[indeed, better, indulge, partying, nonlife, ðÿ¤·ðÿ, », â€â™‚ï¸, year, dangerous, wonâ€™t, he...",noncovid,Topic 10,20-Mar,/r/diabetes/comments/f33pw5/why_why_why_do_i_do_this/fkkmuzs/,0
3,fj2il08,OneFuzzyPillow,2020-02-28 18:49,"For blood sugar, it doesn't seem to affect it. But! You want to stay hydrated for sure, so mak...","[blood, sugar, affect, want, stay, hydrated, sure, sure, drink, water, drink, lot, seltzer, wate...",noncovid,Topic 8,20-Feb,/r/diabetes/comments/f9lhz0/is_drinking_4_diet_cokes_a_day_bad_for_you/fj2il08/,0
4,fj00v3d,Route22,2020-02-28 1:56,"An interesting observation... you might need to try another one. You know, for science.","[interesting, observation, need, try, one, know, science]",noncovid,Topic 5,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj00v3d/,0
5,fj033gg,Person_Something,2020-02-28 2:47,"I donâ€™t really have a problem with ice cream. I eat it quite often, and my bg stays stable. Ma...","[donâ€™t, problem, ice, cream, eat, often, bg, stays, stable, maybe, itâ€™s, personal, reaction,...",noncovid,Topic 1,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj033gg/,0
6,fj08cy7,goddessabove,2020-02-28 4:28,But was it worth it? The shamrock shake was my favorite and now they added Oreos. :( I thought a...,"[worth, shamrock, shake, favorite, added, oreos, thought, getting, taking, bites, know, bother]",noncovid,Topic 8,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj08cy7/,0
7,fj092j6,Grimnex2507,2020-02-28 4:39,I love you for that joke &lt;3,"[love, joke, lt]",noncovid,Topic 4,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj092j6/,0
8,fj0qzir,TheSessionMan,2020-02-28 8:05,You can pre-bolus with MDI too.,"[prebolus, mdi]",noncovid,Topic 3,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj0qzir/,0
9,fj0zmzv,investinlove,2020-02-28 9:19,"My best philosophy is to deny myself nothing, but only take a single bite of the offending food....","[best, philosophy, deny, nothing, single, bite, offending, food, eating, entire, dessert, occur]",noncovid,Topic 1,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj0zmzv/,0


In [13]:
#spacy stopwords
import spacy
sp = spacy.load('en_core_web_sm')

all_spacy_stopwords = sp.Defaults.stop_words
print(len(all_spacy_stopwords))
print(all_spacy_stopwords)

326
{'towards', 'now', 'after', 'seem', 'same', 'only', 'himself', 'former', 'either', 'must', 'would', 'several', 'whenever', 'toward', '’m', 'keep', 'may', 'why', 'does', 'put', 'herself', 'whereupon', 'or', 'above', 'about', 'top', 'if', 'give', 'elsewhere', 'once', 'ever', 'which', 'being', 'until', 'hereafter', 'those', 'something', 'us', 'four', "'d", 'without', 'could', 'used', 'mostly', 'first', 'thereby', 'have', 'against', 'an', 'became', 'what', 'these', 'when', 'hers', 'myself', 'even', 'me', 'two', 'and', 'moreover', 'their', 'whoever', 'too', 'sometimes', 'side', 'done', 'noone', 'afterwards', 'empty', 'others', 'across', 'my', 'might', 'down', 'here', 'with', 'twenty', 'quite', 'somehow', 'hence', 'each', 'out', 'often', 'next', 'very', 'for', 'last', 'therein', 'anyhow', 'sixty', 'all', 'in', 'every', '‘d', 'however', 'do', 'any', 'on', 'three', 'else', 'other', 'latter', 'nevertheless', 'up', 'everyone', 'meanwhile', 'where', 'though', 'beforehand', 'forty', '‘m', 'had

In [14]:
#remove spacy stopwords from text

def remove_spacy_stopwords(text):
    text = [word for word in text if word not in all_spacy_stopwords]
    #print(text)
    return text
    
diabetes_class['text_processed'] = diabetes_class['text_processed'].apply(lambda x: remove_spacy_stopwords(x))
diabetes_class.head(25)

,Post ID,Author,Publish Date,Body,text_processed,classfication,Dominant,month,permalink,Title
0,fkg6fbo,timeflieswhen,2020-03-13 18:58,Did your blood sugar go down finally?,"[blood, sugar, finally]",noncovid,Topic 7,20-Mar,/r/diabetes/comments/ey8ebn/high_blood_sugar_when_fasting/fkg6fbo/,0
1,fqej4ew,GjjWhiteBelt,2020-05-12 11:33,So eat right and exercise hard. Type two is your fault. Get rid of it.,"[eat, right, exercise, hard, type, fault, rid]",noncovid,Topic 8,20-May,/r/diabetes/comments/f212mt/just_diagnosed/fqej4ew/,0
2,fkkmuzs,Frjttr,2020-03-15 8:53,"It is indeed, but better indulge partying than having a non-life ðŸ¤·ðŸ»â€â™‚ï¸ And even once...","[better, indulge, partying, nonlife, ðÿ¤·ðÿ, », â€â™‚ï¸, year, dangerous, wonâ€™t, heal, dama...",noncovid,Topic 10,20-Mar,/r/diabetes/comments/f33pw5/why_why_why_do_i_do_this/fkkmuzs/,0
3,fj2il08,OneFuzzyPillow,2020-02-28 18:49,"For blood sugar, it doesn't seem to affect it. But! You want to stay hydrated for sure, so mak...","[blood, sugar, affect, want, stay, hydrated, sure, sure, drink, water, drink, lot, seltzer, wate...",noncovid,Topic 8,20-Feb,/r/diabetes/comments/f9lhz0/is_drinking_4_diet_cokes_a_day_bad_for_you/fj2il08/,0
4,fj00v3d,Route22,2020-02-28 1:56,"An interesting observation... you might need to try another one. You know, for science.","[interesting, observation, need, try, know, science]",noncovid,Topic 5,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj00v3d/,0
5,fj033gg,Person_Something,2020-02-28 2:47,"I donâ€™t really have a problem with ice cream. I eat it quite often, and my bg stays stable. Ma...","[donâ€™t, problem, ice, cream, eat, bg, stays, stable, maybe, itâ€™s, personal, reaction, drink,...",noncovid,Topic 1,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj033gg/,0
6,fj08cy7,goddessabove,2020-02-28 4:28,But was it worth it? The shamrock shake was my favorite and now they added Oreos. :( I thought a...,"[worth, shamrock, shake, favorite, added, oreos, thought, getting, taking, bites, know, bother]",noncovid,Topic 8,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj08cy7/,0
7,fj092j6,Grimnex2507,2020-02-28 4:39,I love you for that joke &lt;3,"[love, joke, lt]",noncovid,Topic 4,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj092j6/,0
8,fj0qzir,TheSessionMan,2020-02-28 8:05,You can pre-bolus with MDI too.,"[prebolus, mdi]",noncovid,Topic 3,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj0qzir/,0
9,fj0zmzv,investinlove,2020-02-28 9:19,"My best philosophy is to deny myself nothing, but only take a single bite of the offending food....","[best, philosophy, deny, single, bite, offending, food, eating, entire, dessert, occur]",noncovid,Topic 1,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj0zmzv/,0


In [15]:
#lemmitzation
#https://en.wikipedia.org/wiki/Lemmatisation
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

diabetes_class['text_processed'] = diabetes_class['text_processed'].apply(lambda x: lemmatizer(x))
diabetes_class.head(10)

,Post ID,Author,Publish Date,Body,text_processed,classfication,Dominant,month,permalink,Title
0,fkg6fbo,timeflieswhen,2020-03-13 18:58,Did your blood sugar go down finally?,"[blood, sugar, finally]",noncovid,Topic 7,20-Mar,/r/diabetes/comments/ey8ebn/high_blood_sugar_when_fasting/fkg6fbo/,0
1,fqej4ew,GjjWhiteBelt,2020-05-12 11:33,So eat right and exercise hard. Type two is your fault. Get rid of it.,"[eat, right, exercise, hard, type, fault, rid]",noncovid,Topic 8,20-May,/r/diabetes/comments/f212mt/just_diagnosed/fqej4ew/,0
2,fkkmuzs,Frjttr,2020-03-15 8:53,"It is indeed, but better indulge partying than having a non-life ðŸ¤·ðŸ»â€â™‚ï¸ And even once...","[better, indulge, partying, nonlife, ðÿ¤·ðÿ, », â€â™‚ï¸, year, dangerous, wonâ€™t, heal, dama...",noncovid,Topic 10,20-Mar,/r/diabetes/comments/f33pw5/why_why_why_do_i_do_this/fkkmuzs/,0
3,fj2il08,OneFuzzyPillow,2020-02-28 18:49,"For blood sugar, it doesn't seem to affect it. But! You want to stay hydrated for sure, so mak...","[blood, sugar, affect, want, stay, hydrated, sure, sure, drink, water, drink, lot, seltzer, wate...",noncovid,Topic 8,20-Feb,/r/diabetes/comments/f9lhz0/is_drinking_4_diet_cokes_a_day_bad_for_you/fj2il08/,0
4,fj00v3d,Route22,2020-02-28 1:56,"An interesting observation... you might need to try another one. You know, for science.","[interesting, observation, need, try, know, science]",noncovid,Topic 5,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj00v3d/,0
5,fj033gg,Person_Something,2020-02-28 2:47,"I donâ€™t really have a problem with ice cream. I eat it quite often, and my bg stays stable. Ma...","[donâ€™t, problem, ice, cream, eat, bg, stay, stable, maybe, itâ€™s, personal, reaction, drink, ...",noncovid,Topic 1,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj033gg/,0
6,fj08cy7,goddessabove,2020-02-28 4:28,But was it worth it? The shamrock shake was my favorite and now they added Oreos. :( I thought a...,"[worth, shamrock, shake, favorite, added, oreo, thought, getting, taking, bite, know, bother]",noncovid,Topic 8,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj08cy7/,0
7,fj092j6,Grimnex2507,2020-02-28 4:39,I love you for that joke &lt;3,"[love, joke, lt]",noncovid,Topic 4,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj092j6/,0
8,fj0qzir,TheSessionMan,2020-02-28 8:05,You can pre-bolus with MDI too.,"[prebolus, mdi]",noncovid,Topic 3,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj0qzir/,0
9,fj0zmzv,investinlove,2020-02-28 9:19,"My best philosophy is to deny myself nothing, but only take a single bite of the offending food....","[best, philosophy, deny, single, bite, offending, food, eating, entire, dessert, occur]",noncovid,Topic 1,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj0zmzv/,0


In [16]:
#convert text_final to string
diabetes_class['text_processed'] = diabetes_class['text_processed'].astype(str)
diabetes_class

,Post ID,Author,Publish Date,Body,text_processed,classfication,Dominant,month,permalink,Title
0,fkg6fbo,timeflieswhen,2020-03-13 18:58,Did your blood sugar go down finally?,"['blood', 'sugar', 'finally']",noncovid,Topic 7,20-Mar,/r/diabetes/comments/ey8ebn/high_blood_sugar_when_fasting/fkg6fbo/,0
1,fqej4ew,GjjWhiteBelt,2020-05-12 11:33,So eat right and exercise hard. Type two is your fault. Get rid of it.,"['eat', 'right', 'exercise', 'hard', 'type', 'fault', 'rid']",noncovid,Topic 8,20-May,/r/diabetes/comments/f212mt/just_diagnosed/fqej4ew/,0
2,fkkmuzs,Frjttr,2020-03-15 8:53,"It is indeed, but better indulge partying than having a non-life ðŸ¤·ðŸ»â€â™‚ï¸ And even once...","['better', 'indulge', 'partying', 'nonlife', 'ðÿ¤·ðÿ\x8f', '»', 'â€\x8dâ™‚ï¸\x8f', 'year', 'dang...",noncovid,Topic 10,20-Mar,/r/diabetes/comments/f33pw5/why_why_why_do_i_do_this/fkkmuzs/,0
3,fj2il08,OneFuzzyPillow,2020-02-28 18:49,"For blood sugar, it doesn't seem to affect it. But! You want to stay hydrated for sure, so mak...","['blood', 'sugar', 'affect', 'want', 'stay', 'hydrated', 'sure', 'sure', 'drink', 'water', 'drin...",noncovid,Topic 8,20-Feb,/r/diabetes/comments/f9lhz0/is_drinking_4_diet_cokes_a_day_bad_for_you/fj2il08/,0
4,fj00v3d,Route22,2020-02-28 1:56,"An interesting observation... you might need to try another one. You know, for science.","['interesting', 'observation', 'need', 'try', 'know', 'science']",noncovid,Topic 5,20-Feb,/r/diabetes/comments/falppo/had_an_interesting_day_note_to_self_that_shamrock/fj00v3d/,0
...,...,...,...,...,...,...,...,...,...,...
4613,fjhku6k,TeaAndCrackers,2020-03-04 11:27,"I have a list of foods that don't raise my blood sugar, if you'd like to see it. I eat less than...","['list', 'food', 'raise', 'blood', 'sugar', 'youd', 'eat', 'carbs', 'day', 'ac', 'eat', 'meter',...",noncovid,Topic 7,20-Mar,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjhku6k/,0
4614,fjim8u4,dezilove85,2020-03-04 17:12,Would love the list,"['love', 'list']",noncovid,Topic 5,20-Mar,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjim8u4/,0
4615,fjinen3,TeaAndCrackers,2020-03-04 17:24,"ANIMAL PROTEIN:\nBeef, pork, poultry, lamb, game animal ( all parts - no fillers) 85 - 100% fat....","['animal', 'protein', 'beef', 'pork', 'poultry', 'lamb', 'game', 'animal', 'part', 'filler', 'fa...",noncovid,Topic 1,20-Mar,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjinen3/,0
4616,fjinyy3,dezilove85,2020-03-04 17:30,Thank u,"['thank', 'u']",noncovid,Topic 1,20-Mar,/r/diabetes/comments/fde1kh/need_help_lists_ideas/fjinyy3/,0


In [17]:
#test and train data
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(diabetes_class['text_processed'],diabetes_class['classfication'],test_size=0.3)

In [18]:
#encoder
#Encoder = LabelEncoder()
#Train_Y = Encoder.fit_transform(Train_Y)
#Test_Y = Encoder.fit_transform(Test_Y)

In [19]:
#word vectorizor
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(diabetes_class['text_processed'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [20]:
print(Tfidf_vect.vocabulary_)

{'blood': 446, 'sugar': 4269, 'finally': 1598, 'eat': 1364, 'right': 3719, 'exercise': 1480, 'hard': 1870, 'type': 4569, 'fault': 1558, 'rid': 3714, 'better': 398, 'indulge': 2073, 'ðÿ': 4972, 'x8f': 4930, 'x8dâ': 4929, 'year': 4942, 'dangerous': 1114, 'wonâ': 4892, 'heal': 1894, 'damage': 1109, 'caused': 686, 'affect': 112, 'want': 4772, 'stay': 4172, 'hydrated': 1989, 'sure': 4300, 'drink': 1329, 'water': 4796, 'lot': 2438, 'seltzer': 3887, 'got': 1785, 'flavor': 1622, 'count': 1031, 'daily': 1106, 'intake': 2121, 'interesting': 2130, 'observation': 2871, 'need': 2774, 'try': 4550, 'know': 2248, 'science': 3835, 'donâ': 1303, 'problem': 3343, 'ice': 2002, 'cream': 1061, 'bg': 403, 'stable': 4150, 'maybe': 2541, 'itâ': 2162, 'personal': 3115, 'reaction': 3506, 'milk': 2628, 'inject': 2095, 'worth': 4913, 'shake': 3931, 'favorite': 1562, 'added': 66, 'oreo': 2958, 'thought': 4444, 'getting': 1742, 'taking': 4356, 'bite': 424, 'bother': 488, 'love': 2439, 'joke': 2190, 'lt': 2451, 'mdi'

In [21]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
#print(Test_X_Tfidf.getnnz())
#print(predictions_NB)
#print(len(predictions_NB))

# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  74.38494934876991


In [22]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
#predictions_SVM

# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  78.94356005788711


In [23]:
#print classification report
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

print(classification_report(Test_Y,predictions_SVM))

              precision    recall  f1-score   support

       covid       0.85      0.51      0.64       502
    noncovid       0.77      0.95      0.85       880

    accuracy                           0.79      1382
   macro avg       0.81      0.73      0.74      1382
weighted avg       0.80      0.79      0.77      1382



In [24]:
# results
#precision & recall
#print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(Test_Y, predictions_SVM, output_dict=True)
report
#print('positive: ', report['pos'])
#print('negative: ', report['neg'])

{'covid': {'precision': 0.8504983388704319,
  'recall': 0.5099601593625498,
  'f1-score': 0.6376089663760897,
  'support': 502},
 'noncovid': {'precision': 0.7724329324699353,
  'recall': 0.9488636363636364,
  'f1-score': 0.8516063233044366,
  'support': 880},
 'accuracy': 0.7894356005788712,
 'macro avg': {'precision': 0.8114656356701836,
  'recall': 0.729411897863093,
  'f1-score': 0.7446076448402632,
  'support': 1382},
 'weighted avg': {'precision': 0.8007895417413169,
  'recall': 0.7894356005788712,
  'f1-score': 0.7738735641307535,
  'support': 1382}}

In [25]:
print('covid: ', report['covid'])
print('noncovid: ', report['noncovid'])

covid:  {'precision': 0.8504983388704319, 'recall': 0.5099601593625498, 'f1-score': 0.6376089663760897, 'support': 502}
noncovid:  {'precision': 0.7724329324699353, 'recall': 0.9488636363636364, 'f1-score': 0.8516063233044366, 'support': 880}


In [26]:
data = ['i had a virtual visit', 'lockdown just sucks', 'i got my A1c down!!!', 'the gyms are closed',  
         'lets go to mall tomorrow', 'i am very proud of you', 'thank you', 'i need to buy toilet paper', 'this pandemic is making me depressed','we need to quarantine', 'i got pfizer',
       'go see a doctor ASAP', 'you should eat more protein', 'metformin shortages', 'i feel nauseaus']

df_test = pd.DataFrame(data, columns = ['text'])

In [27]:
#convert to lower
df_test['text_processed'] = df_test['text'].map(lambda x: x.lower())
df_test

,text,text_processed
0,i had a virtual visit,i had a virtual visit
1,lockdown just sucks,lockdown just sucks
2,i got my A1c down!!!,i got my a1c down!!!
3,the gyms are closed,the gyms are closed
4,lets go to mall tomorrow,lets go to mall tomorrow
5,i am very proud of you,i am very proud of you
6,thank you,thank you
7,i need to buy toilet paper,i need to buy toilet paper
8,this pandemic is making me depressed,this pandemic is making me depressed
9,we need to quarantine,we need to quarantine


In [28]:
#remove gensim stopwords
df_test['text_processed'] = df_test['text_processed'].map(lambda x: remove_stopwords(x))
df_test

,text,text_processed
0,i had a virtual visit,virtual visit
1,lockdown just sucks,lockdown sucks
2,i got my A1c down!!!,got a1c down!!!
3,the gyms are closed,gyms closed
4,lets go to mall tomorrow,lets mall tomorrow
5,i am very proud of you,proud
6,thank you,thank
7,i need to buy toilet paper,need buy toilet paper
8,this pandemic is making me depressed,pandemic making depressed
9,we need to quarantine,need quarantine


In [29]:
#remove puncuation
df_test['text_processed'] = df_test['text_processed'].map(lambda x: remove_punct(x))
df_test

,text,text_processed
0,i had a virtual visit,virtual visit
1,lockdown just sucks,lockdown sucks
2,i got my A1c down!!!,got ac down
3,the gyms are closed,gyms closed
4,lets go to mall tomorrow,lets mall tomorrow
5,i am very proud of you,proud
6,thank you,thank
7,i need to buy toilet paper,need buy toilet paper
8,this pandemic is making me depressed,pandemic making depressed
9,we need to quarantine,need quarantine


In [30]:
#tokenize text
df_test['text_processed'] = df_test['text_processed'].map(lambda x: word_tokenize(x))
df_test

,text,text_processed
0,i had a virtual visit,"[virtual, visit]"
1,lockdown just sucks,"[lockdown, sucks]"
2,i got my A1c down!!!,"[got, ac, down]"
3,the gyms are closed,"[gyms, closed]"
4,lets go to mall tomorrow,"[lets, mall, tomorrow]"
5,i am very proud of you,[proud]
6,thank you,[thank]
7,i need to buy toilet paper,"[need, buy, toilet, paper]"
8,this pandemic is making me depressed,"[pandemic, making, depressed]"
9,we need to quarantine,"[need, quarantine]"


In [31]:
#lemmitzation
df_test['text_processed']= df_test['text_processed'].map(lambda x: lemmatizer(x))
df_test

,text,text_processed
0,i had a virtual visit,"[virtual, visit]"
1,lockdown just sucks,"[lockdown, suck]"
2,i got my A1c down!!!,"[got, ac, down]"
3,the gyms are closed,"[gym, closed]"
4,lets go to mall tomorrow,"[let, mall, tomorrow]"
5,i am very proud of you,[proud]
6,thank you,[thank]
7,i need to buy toilet paper,"[need, buy, toilet, paper]"
8,this pandemic is making me depressed,"[pandemic, making, depressed]"
9,we need to quarantine,"[need, quarantine]"


In [32]:
#convert dataframe to list
df_list = df_test['text_processed'].to_list()
df_list

[['virtual', 'visit'],
 ['lockdown', 'suck'],
 ['got', 'ac', 'down'],
 ['gym', 'closed'],
 ['let', 'mall', 'tomorrow'],
 ['proud'],
 ['thank'],
 ['need', 'buy', 'toilet', 'paper'],
 ['pandemic', 'making', 'depressed'],
 ['need', 'quarantine'],
 ['got', 'pfizer'],
 ['doctor', 'asap'],
 ['eat', 'protein'],
 ['metformin', 'shortage'],
 ['feel', 'nauseaus']]

In [33]:
df_list_corrected = [" ".join(x) for x in df_list]
df_list_corrected

['virtual visit',
 'lockdown suck',
 'got ac down',
 'gym closed',
 'let mall tomorrow',
 'proud',
 'thank',
 'need buy toilet paper',
 'pandemic making depressed',
 'need quarantine',
 'got pfizer',
 'doctor asap',
 'eat protein',
 'metformin shortage',
 'feel nauseaus']

In [34]:
#word vectorizor
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(diabetes_class['text_processed'])
df_Tfidf = Tfidf_vect.transform(df_list_corrected)
df_Tfidf

<15x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 30 stored elements in Compressed Sparse Row format>

In [35]:
# predict the labels on validation dataset
Naive.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_NB = Naive.predict(df_Tfidf)
print(df_Tfidf.getnnz())
print(predictions_NB)
print(len(predictions_NB))
print(df_list_corrected)

30
['covid' 'covid' 'noncovid' 'covid' 'noncovid' 'noncovid' 'noncovid'
 'covid' 'covid' 'covid' 'covid' 'noncovid' 'noncovid' 'covid' 'noncovid']
15
['virtual visit', 'lockdown suck', 'got ac down', 'gym closed', 'let mall tomorrow', 'proud', 'thank', 'need buy toilet paper', 'pandemic making depressed', 'need quarantine', 'got pfizer', 'doctor asap', 'eat protein', 'metformin shortage', 'feel nauseaus']


In [36]:
# predict the labels on validation dataset
SVM.fit(Train_X_Tfidf,Train_Y)
predictions_SVM = SVM.predict(df_Tfidf)
print(df_Tfidf.getnnz())
print(predictions_SVM)
print(len(predictions_SVM))
print(df_list_corrected)

30
['covid' 'covid' 'noncovid' 'covid' 'covid' 'noncovid' 'noncovid' 'covid'
 'covid' 'covid' 'covid' 'noncovid' 'noncovid' 'covid' 'noncovid']
15
['virtual visit', 'lockdown suck', 'got ac down', 'gym closed', 'let mall tomorrow', 'proud', 'thank', 'need buy toilet paper', 'pandemic making depressed', 'need quarantine', 'got pfizer', 'doctor asap', 'eat protein', 'metformin shortage', 'feel nauseaus']


In [37]:
df_test['classification'] = predictions_SVM
df_test

,text,text_processed,classification
0,i had a virtual visit,"[virtual, visit]",covid
1,lockdown just sucks,"[lockdown, suck]",covid
2,i got my A1c down!!!,"[got, ac, down]",noncovid
3,the gyms are closed,"[gym, closed]",covid
4,lets go to mall tomorrow,"[let, mall, tomorrow]",covid
5,i am very proud of you,[proud],noncovid
6,thank you,[thank],noncovid
7,i need to buy toilet paper,"[need, buy, toilet, paper]",covid
8,this pandemic is making me depressed,"[pandemic, making, depressed]",covid
9,we need to quarantine,"[need, quarantine]",covid


In [56]:
#load testing dataset
diabetes_test = pd.read_csv('diabetes_class_validation.csv')
diabetes_test

,Post ID,Author,Publish Date,Body,Dominant,month,classfication,permalink,Title
0,fdybas,Christiancarrion,2020-03-05 9:05,I have Type 2. Looking to get off my ass (again ðŸ˜’ðŸ¥´ðŸ˜‰) and refresh my diet. What are some...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/,Favorite snacks?
1,fjkjx8v,kmkmrod,2020-03-05 9:11,"Jerky, pistachios (many other nuts but those are favorite), apples, 2g yogurt with seed granola,...",Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjkjx8v/,0
2,fjkw6jm,derioderio,2020-03-05 11:05,* Homemade jerky: no sugar = no carbs\n* Homemade yogurt: no added sugar = much less carbs than ...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjkw6jm/,0
3,fjld033,alan_s,2020-03-05 13:41,PB in the channel of a celery chunk. Salami sticks. Boiled eggs mashed with a little mayo and cu...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjld033/,0
4,fjmjk41,rharmelink,2020-03-05 21:27,"My ""go to"" for hunger is just a cheesewich. My simplest is just 2 slices of Canadian bacon on a ...",Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjmjk41/,0
...,...,...,...,...,...,...,...,...,...
44365,gojn4i5,didyouwoof,2021-02-23 19:47,"OPâ€™s complaining about a bitter taste, though, not a sour taste.",Topic 1,21-Feb,NaN,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gojn4i5/,0
44366,gonephk,alan_s,2021-02-24 16:53,"Whatever way you defrost them and whatever they taste like, test an hour after eating them to se...",Topic 6,21-Feb,NaN,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gonephk/,0
44367,gol92t4,SavingMyFoot_DH,2021-02-24 7:22,[,Topic 1,21-Feb,NaN,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gol92t4/,0
44368,gomkg3y,hidflect1,2021-02-24 12:42,"Short term, drinking low carb alcohol lowers your blood sugar because it interferes with your li...",Topic 10,21-Feb,NaN,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gomkg3y/,0


In [57]:
#remove any blank texts
diabetes_test = diabetes_test[diabetes_test['Body'].notna()]
diabetes_test

,Post ID,Author,Publish Date,Body,Dominant,month,classfication,permalink,Title
0,fdybas,Christiancarrion,2020-03-05 9:05,I have Type 2. Looking to get off my ass (again ðŸ˜’ðŸ¥´ðŸ˜‰) and refresh my diet. What are some...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/,Favorite snacks?
1,fjkjx8v,kmkmrod,2020-03-05 9:11,"Jerky, pistachios (many other nuts but those are favorite), apples, 2g yogurt with seed granola,...",Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjkjx8v/,0
2,fjkw6jm,derioderio,2020-03-05 11:05,* Homemade jerky: no sugar = no carbs\n* Homemade yogurt: no added sugar = much less carbs than ...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjkw6jm/,0
3,fjld033,alan_s,2020-03-05 13:41,PB in the channel of a celery chunk. Salami sticks. Boiled eggs mashed with a little mayo and cu...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjld033/,0
4,fjmjk41,rharmelink,2020-03-05 21:27,"My ""go to"" for hunger is just a cheesewich. My simplest is just 2 slices of Canadian bacon on a ...",Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjmjk41/,0
...,...,...,...,...,...,...,...,...,...
44365,gojn4i5,didyouwoof,2021-02-23 19:47,"OPâ€™s complaining about a bitter taste, though, not a sour taste.",Topic 1,21-Feb,NaN,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gojn4i5/,0
44366,gonephk,alan_s,2021-02-24 16:53,"Whatever way you defrost them and whatever they taste like, test an hour after eating them to se...",Topic 6,21-Feb,NaN,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gonephk/,0
44367,gol92t4,SavingMyFoot_DH,2021-02-24 7:22,[,Topic 1,21-Feb,NaN,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gol92t4/,0
44368,gomkg3y,hidflect1,2021-02-24 12:42,"Short term, drinking low carb alcohol lowers your blood sugar because it interferes with your li...",Topic 10,21-Feb,NaN,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gomkg3y/,0


In [58]:
#convert to lower
diabetes_test['text_processed'] = diabetes_test['Body'].map(lambda x: x.lower())
diabetes_test

<ipython-input-58-45848df1974d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diabetes_test['text_processed'] = diabetes_test['Body'].map(lambda x: x.lower())


,Post ID,Author,Publish Date,Body,Dominant,month,classfication,permalink,Title,text_processed
0,fdybas,Christiancarrion,2020-03-05 9:05,I have Type 2. Looking to get off my ass (again ðŸ˜’ðŸ¥´ðŸ˜‰) and refresh my diet. What are some...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/,Favorite snacks?,i have type 2. looking to get off my ass (again ðÿ˜’ðÿ¥´ðÿ˜‰) and refresh my diet. what are some...
1,fjkjx8v,kmkmrod,2020-03-05 9:11,"Jerky, pistachios (many other nuts but those are favorite), apples, 2g yogurt with seed granola,...",Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjkjx8v/,0,"jerky, pistachios (many other nuts but those are favorite), apples, 2g yogurt with seed granola,..."
2,fjkw6jm,derioderio,2020-03-05 11:05,* Homemade jerky: no sugar = no carbs\n* Homemade yogurt: no added sugar = much less carbs than ...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjkw6jm/,0,* homemade jerky: no sugar = no carbs\n* homemade yogurt: no added sugar = much less carbs than ...
3,fjld033,alan_s,2020-03-05 13:41,PB in the channel of a celery chunk. Salami sticks. Boiled eggs mashed with a little mayo and cu...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjld033/,0,pb in the channel of a celery chunk. salami sticks. boiled eggs mashed with a little mayo and cu...
4,fjmjk41,rharmelink,2020-03-05 21:27,"My ""go to"" for hunger is just a cheesewich. My simplest is just 2 slices of Canadian bacon on a ...",Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjmjk41/,0,"my ""go to"" for hunger is just a cheesewich. my simplest is just 2 slices of canadian bacon on a ..."
...,...,...,...,...,...,...,...,...,...,...
44365,gojn4i5,didyouwoof,2021-02-23 19:47,"OPâ€™s complaining about a bitter taste, though, not a sour taste.",Topic 1,21-Feb,NaN,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gojn4i5/,0,"opâ€™s complaining about a bitter taste, though, not a sour taste."
44366,gonephk,alan_s,2021-02-24 16:53,"Whatever way you defrost them and whatever they taste like, test an hour after eating them to se...",Topic 6,21-Feb,NaN,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gonephk/,0,"whatever way you defrost them and whatever they taste like, test an hour after eating them to se..."
44367,gol92t4,SavingMyFoot_DH,2021-02-24 7:22,[,Topic 1,21-Feb,NaN,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gol92t4/,0,[
44368,gomkg3y,hidflect1,2021-02-24 12:42,"Short term, drinking low carb alcohol lowers your blood sugar because it interferes with your li...",Topic 10,21-Feb,NaN,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gomkg3y/,0,"short term, drinking low carb alcohol lowers your blood sugar because it interferes with your li..."


In [59]:
#remove gensim stopwords
diabetes_test['text_processed'] = diabetes_test['text_processed'].map(lambda x: remove_stopwords(x))
diabetes_test

<ipython-input-59-9d175d4529f0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diabetes_test['text_processed'] = diabetes_test['text_processed'].map(lambda x: remove_stopwords(x))


,Post ID,Author,Publish Date,Body,Dominant,month,classfication,permalink,Title,text_processed
0,fdybas,Christiancarrion,2020-03-05 9:05,I have Type 2. Looking to get off my ass (again ðŸ˜’ðŸ¥´ðŸ˜‰) and refresh my diet. What are some...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/,Favorite snacks?,type 2. looking ass (again ðÿ˜’ðÿ¥´ðÿ˜‰) refresh diet. favorite diabetic-friendly snacks?
1,fjkjx8v,kmkmrod,2020-03-05 9:11,"Jerky, pistachios (many other nuts but those are favorite), apples, 2g yogurt with seed granola,...",Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjkjx8v/,0,"jerky, pistachios (many nuts favorite), apples, 2g yogurt seed granola, celery peppers hummus, p..."
2,fjkw6jm,derioderio,2020-03-05 11:05,* Homemade jerky: no sugar = no carbs\n* Homemade yogurt: no added sugar = much less carbs than ...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjkw6jm/,0,* homemade jerky: sugar = carbs * homemade yogurt: added sugar = carbs store bought yogurt * hom...
3,fjld033,alan_s,2020-03-05 13:41,PB in the channel of a celery chunk. Salami sticks. Boiled eggs mashed with a little mayo and cu...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjld033/,0,pb channel celery chunk. salami sticks. boiled eggs mashed little mayo curry powder. ideas: [sna...
4,fjmjk41,rharmelink,2020-03-05 21:27,"My ""go to"" for hunger is just a cheesewich. My simplest is just 2 slices of Canadian bacon on a ...",Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjmjk41/,0,"""go to"" hunger cheesewich. simplest 2 slices canadian bacon slice pepper jack cheese. fold eat. ..."
...,...,...,...,...,...,...,...,...,...,...
44365,gojn4i5,didyouwoof,2021-02-23 19:47,"OPâ€™s complaining about a bitter taste, though, not a sour taste.",Topic 1,21-Feb,NaN,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gojn4i5/,0,"opâ€™s complaining bitter taste, though, sour taste."
44366,gonephk,alan_s,2021-02-24 16:53,"Whatever way you defrost them and whatever they taste like, test an hour after eating them to se...",Topic 6,21-Feb,NaN,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gonephk/,0,"way defrost taste like, test hour eating true effects blood glucose levels. &gt;i've im diabetic..."
44367,gol92t4,SavingMyFoot_DH,2021-02-24 7:22,[,Topic 1,21-Feb,NaN,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gol92t4/,0,[
44368,gomkg3y,hidflect1,2021-02-24 12:42,"Short term, drinking low carb alcohol lowers your blood sugar because it interferes with your li...",Topic 10,21-Feb,NaN,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gomkg3y/,0,"short term, drinking low carb alcohol lowers blood sugar interferes liver's production glucose. ..."


In [60]:
#remove puncuation
diabetes_test['text_processed'] = diabetes_test['text_processed'].map(lambda x: remove_punct(x))
diabetes_test

<ipython-input-60-e06e3d227336>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diabetes_test['text_processed'] = diabetes_test['text_processed'].map(lambda x: remove_punct(x))


,Post ID,Author,Publish Date,Body,Dominant,month,classfication,permalink,Title,text_processed
0,fdybas,Christiancarrion,2020-03-05 9:05,I have Type 2. Looking to get off my ass (again ðŸ˜’ðŸ¥´ðŸ˜‰) and refresh my diet. What are some...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/,Favorite snacks?,type looking ass again ðÿ˜’ðÿ¥´ðÿ˜‰ refresh diet favorite diabeticfriendly snacks
1,fjkjx8v,kmkmrod,2020-03-05 9:11,"Jerky, pistachios (many other nuts but those are favorite), apples, 2g yogurt with seed granola,...",Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjkjx8v/,0,jerky pistachios many nuts favorite apples g yogurt seed granola celery peppers hummus pepperoni...
2,fjkw6jm,derioderio,2020-03-05 11:05,* Homemade jerky: no sugar = no carbs\n* Homemade yogurt: no added sugar = much less carbs than ...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjkw6jm/,0,homemade jerky sugar carbs homemade yogurt added sugar carbs store bought yogurt homemade i...
3,fjld033,alan_s,2020-03-05 13:41,PB in the channel of a celery chunk. Salami sticks. Boiled eggs mashed with a little mayo and cu...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjld033/,0,pb channel celery chunk salami sticks boiled eggs mashed little mayo curry powder ideas snacks
4,fjmjk41,rharmelink,2020-03-05 21:27,"My ""go to"" for hunger is just a cheesewich. My simplest is just 2 slices of Canadian bacon on a ...",Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjmjk41/,0,go to hunger cheesewich simplest slices canadian bacon slice pepper jack cheese fold eat im cra...
...,...,...,...,...,...,...,...,...,...,...
44365,gojn4i5,didyouwoof,2021-02-23 19:47,"OPâ€™s complaining about a bitter taste, though, not a sour taste.",Topic 1,21-Feb,NaN,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gojn4i5/,0,opâ€™s complaining bitter taste though sour taste
44366,gonephk,alan_s,2021-02-24 16:53,"Whatever way you defrost them and whatever they taste like, test an hour after eating them to se...",Topic 6,21-Feb,NaN,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gonephk/,0,way defrost taste like test hour eating true effects blood glucose levels gtive im diabetic clic...
44367,gol92t4,SavingMyFoot_DH,2021-02-24 7:22,[,Topic 1,21-Feb,NaN,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gol92t4/,0,
44368,gomkg3y,hidflect1,2021-02-24 12:42,"Short term, drinking low carb alcohol lowers your blood sugar because it interferes with your li...",Topic 10,21-Feb,NaN,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gomkg3y/,0,short term drinking low carb alcohol lowers blood sugar interferes livers production glucose alc...


In [61]:
#tokenize text
diabetes_test['text_processed'] = diabetes_test['text_processed'].map(lambda x: word_tokenize(x))
diabetes_test

<ipython-input-61-f4cce7fb3e20>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diabetes_test['text_processed'] = diabetes_test['text_processed'].map(lambda x: word_tokenize(x))


,Post ID,Author,Publish Date,Body,Dominant,month,classfication,permalink,Title,text_processed
0,fdybas,Christiancarrion,2020-03-05 9:05,I have Type 2. Looking to get off my ass (again ðŸ˜’ðŸ¥´ðŸ˜‰) and refresh my diet. What are some...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/,Favorite snacks?,"[type, looking, ass, again, ðÿ˜, ’, ðÿ¥´ðÿ˜‰, refresh, diet, favorite, diabeticfriendly, snacks]"
1,fjkjx8v,kmkmrod,2020-03-05 9:11,"Jerky, pistachios (many other nuts but those are favorite), apples, 2g yogurt with seed granola,...",Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjkjx8v/,0,"[jerky, pistachios, many, nuts, favorite, apples, g, yogurt, seed, granola, celery, peppers, hum..."
2,fjkw6jm,derioderio,2020-03-05 11:05,* Homemade jerky: no sugar = no carbs\n* Homemade yogurt: no added sugar = much less carbs than ...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjkw6jm/,0,"[homemade, jerky, sugar, carbs, homemade, yogurt, added, sugar, carbs, store, bought, yogurt, ho..."
3,fjld033,alan_s,2020-03-05 13:41,PB in the channel of a celery chunk. Salami sticks. Boiled eggs mashed with a little mayo and cu...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjld033/,0,"[pb, channel, celery, chunk, salami, sticks, boiled, eggs, mashed, little, mayo, curry, powder, ..."
4,fjmjk41,rharmelink,2020-03-05 21:27,"My ""go to"" for hunger is just a cheesewich. My simplest is just 2 slices of Canadian bacon on a ...",Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjmjk41/,0,"[go, to, hunger, cheesewich, simplest, slices, canadian, bacon, slice, pepper, jack, cheese, fol..."
...,...,...,...,...,...,...,...,...,...,...
44365,gojn4i5,didyouwoof,2021-02-23 19:47,"OPâ€™s complaining about a bitter taste, though, not a sour taste.",Topic 1,21-Feb,NaN,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gojn4i5/,0,"[opâ€™s, complaining, bitter, taste, though, sour, taste]"
44366,gonephk,alan_s,2021-02-24 16:53,"Whatever way you defrost them and whatever they taste like, test an hour after eating them to se...",Topic 6,21-Feb,NaN,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gonephk/,0,"[way, defrost, taste, like, test, hour, eating, true, effects, blood, glucose, levels, gtive, im..."
44367,gol92t4,SavingMyFoot_DH,2021-02-24 7:22,[,Topic 1,21-Feb,NaN,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gol92t4/,0,[]
44368,gomkg3y,hidflect1,2021-02-24 12:42,"Short term, drinking low carb alcohol lowers your blood sugar because it interferes with your li...",Topic 10,21-Feb,NaN,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gomkg3y/,0,"[short, term, drinking, low, carb, alcohol, lowers, blood, sugar, interferes, livers, production..."


In [62]:
#lemmitzation
diabetes_test['text_processed']= diabetes_test['text_processed'].map(lambda x: lemmatizer(x))
diabetes_test

<ipython-input-62-8bdbe6a49c28>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diabetes_test['text_processed']= diabetes_test['text_processed'].map(lambda x: lemmatizer(x))


,Post ID,Author,Publish Date,Body,Dominant,month,classfication,permalink,Title,text_processed
0,fdybas,Christiancarrion,2020-03-05 9:05,I have Type 2. Looking to get off my ass (again ðŸ˜’ðŸ¥´ðŸ˜‰) and refresh my diet. What are some...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/,Favorite snacks?,"[type, looking, as, again, ðÿ˜, ’, ðÿ¥´ðÿ˜‰, refresh, diet, favorite, diabeticfriendly, snack]"
1,fjkjx8v,kmkmrod,2020-03-05 9:11,"Jerky, pistachios (many other nuts but those are favorite), apples, 2g yogurt with seed granola,...",Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjkjx8v/,0,"[jerky, pistachio, many, nut, favorite, apple, g, yogurt, seed, granola, celery, pepper, hummus,..."
2,fjkw6jm,derioderio,2020-03-05 11:05,* Homemade jerky: no sugar = no carbs\n* Homemade yogurt: no added sugar = much less carbs than ...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjkw6jm/,0,"[homemade, jerky, sugar, carbs, homemade, yogurt, added, sugar, carbs, store, bought, yogurt, ho..."
3,fjld033,alan_s,2020-03-05 13:41,PB in the channel of a celery chunk. Salami sticks. Boiled eggs mashed with a little mayo and cu...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjld033/,0,"[pb, channel, celery, chunk, salami, stick, boiled, egg, mashed, little, mayo, curry, powder, id..."
4,fjmjk41,rharmelink,2020-03-05 21:27,"My ""go to"" for hunger is just a cheesewich. My simplest is just 2 slices of Canadian bacon on a ...",Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjmjk41/,0,"[go, to, hunger, cheesewich, simplest, slice, canadian, bacon, slice, pepper, jack, cheese, fold..."
...,...,...,...,...,...,...,...,...,...,...
44365,gojn4i5,didyouwoof,2021-02-23 19:47,"OPâ€™s complaining about a bitter taste, though, not a sour taste.",Topic 1,21-Feb,NaN,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gojn4i5/,0,"[opâ€™s, complaining, bitter, taste, though, sour, taste]"
44366,gonephk,alan_s,2021-02-24 16:53,"Whatever way you defrost them and whatever they taste like, test an hour after eating them to se...",Topic 6,21-Feb,NaN,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gonephk/,0,"[way, defrost, taste, like, test, hour, eating, true, effect, blood, glucose, level, gtive, im, ..."
44367,gol92t4,SavingMyFoot_DH,2021-02-24 7:22,[,Topic 1,21-Feb,NaN,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gol92t4/,0,[]
44368,gomkg3y,hidflect1,2021-02-24 12:42,"Short term, drinking low carb alcohol lowers your blood sugar because it interferes with your li...",Topic 10,21-Feb,NaN,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gomkg3y/,0,"[short, term, drinking, low, carb, alcohol, lower, blood, sugar, interferes, liver, production, ..."


In [63]:
#convert dataframe to list
diabetes_list = diabetes_test['text_processed'].to_list()
#diabetes_list

In [64]:
diabetes_list_corrected = [" ".join(x) for x in diabetes_list]
#diabetes_list_corrected

In [65]:
#word vectorizor
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(diabetes_class['text_processed'])
diabetes_Tfidf = Tfidf_vect.transform(diabetes_list_corrected)
diabetes_Tfidf

<44293x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 827812 stored elements in Compressed Sparse Row format>

In [66]:
# predict the labels on validation dataset
Naive.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_NB = Naive.predict(diabetes_Tfidf)
#print(diabetes_Tfidf.getnnz())
#print(predictions_NB)
#print(len(predictions_NB))
#print(diabetes_list_corrected)

In [67]:
# predict the labels on validation dataset
SVM.fit(Train_X_Tfidf,Train_Y)
predictions_SVM = SVM.predict(diabetes_Tfidf)
#print(diabetes_Tfidf.getnnz())
#print(predictions_SVM)
#print(len(predictions_SVM))
#print(diabetes_list_corrected)

In [68]:
diabetes_test['classification'] = predictions_SVM
diabetes_test

<ipython-input-68-1e64357e73ae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diabetes_test['classification'] = predictions_SVM


,Post ID,Author,Publish Date,Body,Dominant,month,classfication,permalink,Title,text_processed,classification
0,fdybas,Christiancarrion,2020-03-05 9:05,I have Type 2. Looking to get off my ass (again ðŸ˜’ðŸ¥´ðŸ˜‰) and refresh my diet. What are some...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/,Favorite snacks?,"[type, looking, as, again, ðÿ˜, ’, ðÿ¥´ðÿ˜‰, refresh, diet, favorite, diabeticfriendly, snack]",noncovid
1,fjkjx8v,kmkmrod,2020-03-05 9:11,"Jerky, pistachios (many other nuts but those are favorite), apples, 2g yogurt with seed granola,...",Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjkjx8v/,0,"[jerky, pistachio, many, nut, favorite, apple, g, yogurt, seed, granola, celery, pepper, hummus,...",noncovid
2,fjkw6jm,derioderio,2020-03-05 11:05,* Homemade jerky: no sugar = no carbs\n* Homemade yogurt: no added sugar = much less carbs than ...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjkw6jm/,0,"[homemade, jerky, sugar, carbs, homemade, yogurt, added, sugar, carbs, store, bought, yogurt, ho...",noncovid
3,fjld033,alan_s,2020-03-05 13:41,PB in the channel of a celery chunk. Salami sticks. Boiled eggs mashed with a little mayo and cu...,Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjld033/,0,"[pb, channel, celery, chunk, salami, stick, boiled, egg, mashed, little, mayo, curry, powder, id...",noncovid
4,fjmjk41,rharmelink,2020-03-05 21:27,"My ""go to"" for hunger is just a cheesewich. My simplest is just 2 slices of Canadian bacon on a ...",Topic 1,20-Mar,NaN,/r/diabetes/comments/fdybas/favorite_snacks/fjmjk41/,0,"[go, to, hunger, cheesewich, simplest, slice, canadian, bacon, slice, pepper, jack, cheese, fold...",noncovid
...,...,...,...,...,...,...,...,...,...,...,...
44365,gojn4i5,didyouwoof,2021-02-23 19:47,"OPâ€™s complaining about a bitter taste, though, not a sour taste.",Topic 1,21-Feb,NaN,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gojn4i5/,0,"[opâ€™s, complaining, bitter, taste, though, sour, taste]",noncovid
44366,gonephk,alan_s,2021-02-24 16:53,"Whatever way you defrost them and whatever they taste like, test an hour after eating them to se...",Topic 6,21-Feb,NaN,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gonephk/,0,"[way, defrost, taste, like, test, hour, eating, true, effect, blood, glucose, level, gtive, im, ...",noncovid
44367,gol92t4,SavingMyFoot_DH,2021-02-24 7:22,[,Topic 1,21-Feb,NaN,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gol92t4/,0,[],noncovid
44368,gomkg3y,hidflect1,2021-02-24 12:42,"Short term, drinking low carb alcohol lowers your blood sugar because it interferes with your li...",Topic 10,21-Feb,NaN,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gomkg3y/,0,"[short, term, drinking, low, carb, alcohol, lower, blood, sugar, interferes, liver, production, ...",noncovid


In [70]:
diabetes_test = diabetes_test.drop(['classfication', 'text_processed'], axis = 1)
diabetes_test

,Post ID,Author,Publish Date,Body,Dominant,month,permalink,Title,classification
0,fdybas,Christiancarrion,2020-03-05 9:05,I have Type 2. Looking to get off my ass (again ðŸ˜’ðŸ¥´ðŸ˜‰) and refresh my diet. What are some...,Topic 1,20-Mar,/r/diabetes/comments/fdybas/favorite_snacks/,Favorite snacks?,noncovid
1,fjkjx8v,kmkmrod,2020-03-05 9:11,"Jerky, pistachios (many other nuts but those are favorite), apples, 2g yogurt with seed granola,...",Topic 1,20-Mar,/r/diabetes/comments/fdybas/favorite_snacks/fjkjx8v/,0,noncovid
2,fjkw6jm,derioderio,2020-03-05 11:05,* Homemade jerky: no sugar = no carbs\n* Homemade yogurt: no added sugar = much less carbs than ...,Topic 1,20-Mar,/r/diabetes/comments/fdybas/favorite_snacks/fjkw6jm/,0,noncovid
3,fjld033,alan_s,2020-03-05 13:41,PB in the channel of a celery chunk. Salami sticks. Boiled eggs mashed with a little mayo and cu...,Topic 1,20-Mar,/r/diabetes/comments/fdybas/favorite_snacks/fjld033/,0,noncovid
4,fjmjk41,rharmelink,2020-03-05 21:27,"My ""go to"" for hunger is just a cheesewich. My simplest is just 2 slices of Canadian bacon on a ...",Topic 1,20-Mar,/r/diabetes/comments/fdybas/favorite_snacks/fjmjk41/,0,noncovid
...,...,...,...,...,...,...,...,...,...
44365,gojn4i5,didyouwoof,2021-02-23 19:47,"OPâ€™s complaining about a bitter taste, though, not a sour taste.",Topic 1,21-Feb,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gojn4i5/,0,noncovid
44366,gonephk,alan_s,2021-02-24 16:53,"Whatever way you defrost them and whatever they taste like, test an hour after eating them to se...",Topic 6,21-Feb,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gonephk/,0,noncovid
44367,gol92t4,SavingMyFoot_DH,2021-02-24 7:22,[,Topic 1,21-Feb,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gol92t4/,0,noncovid
44368,gomkg3y,hidflect1,2021-02-24 12:42,"Short term, drinking low carb alcohol lowers your blood sugar because it interferes with your li...",Topic 10,21-Feb,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gomkg3y/,0,noncovid


In [72]:
list(diabetes_test.columns)

['Post ID',
 'Author',
 'Publish Date',
 'Body',
 'Dominant',
 'month',
 'permalink',
 'Title',
 'classification']

In [78]:
diabetes_test = diabetes_test[['Post ID', 'Author', 'Publish Date','Body', 'Dominant', 'month','classification','permalink','Title']]

In [79]:
diabetes_test

,Post ID,Author,Publish Date,Body,Dominant,month,classification,permalink,Title
0,fdybas,Christiancarrion,2020-03-05 9:05,I have Type 2. Looking to get off my ass (again ðŸ˜’ðŸ¥´ðŸ˜‰) and refresh my diet. What are some...,Topic 1,20-Mar,noncovid,/r/diabetes/comments/fdybas/favorite_snacks/,Favorite snacks?
1,fjkjx8v,kmkmrod,2020-03-05 9:11,"Jerky, pistachios (many other nuts but those are favorite), apples, 2g yogurt with seed granola,...",Topic 1,20-Mar,noncovid,/r/diabetes/comments/fdybas/favorite_snacks/fjkjx8v/,0
2,fjkw6jm,derioderio,2020-03-05 11:05,* Homemade jerky: no sugar = no carbs\n* Homemade yogurt: no added sugar = much less carbs than ...,Topic 1,20-Mar,noncovid,/r/diabetes/comments/fdybas/favorite_snacks/fjkw6jm/,0
3,fjld033,alan_s,2020-03-05 13:41,PB in the channel of a celery chunk. Salami sticks. Boiled eggs mashed with a little mayo and cu...,Topic 1,20-Mar,noncovid,/r/diabetes/comments/fdybas/favorite_snacks/fjld033/,0
4,fjmjk41,rharmelink,2020-03-05 21:27,"My ""go to"" for hunger is just a cheesewich. My simplest is just 2 slices of Canadian bacon on a ...",Topic 1,20-Mar,noncovid,/r/diabetes/comments/fdybas/favorite_snacks/fjmjk41/,0
...,...,...,...,...,...,...,...,...,...
44365,gojn4i5,didyouwoof,2021-02-23 19:47,"OPâ€™s complaining about a bitter taste, though, not a sour taste.",Topic 1,21-Feb,noncovid,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gojn4i5/,0
44366,gonephk,alan_s,2021-02-24 16:53,"Whatever way you defrost them and whatever they taste like, test an hour after eating them to se...",Topic 6,21-Feb,noncovid,/r/type2diabetes/comments/lqto9y/mixed_berries_for_a_treat/gonephk/,0
44367,gol92t4,SavingMyFoot_DH,2021-02-24 7:22,[,Topic 1,21-Feb,noncovid,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gol92t4/,0
44368,gomkg3y,hidflect1,2021-02-24 12:42,"Short term, drinking low carb alcohol lowers your blood sugar because it interferes with your li...",Topic 10,21-Feb,noncovid,/r/type2diabetes/comments/lrfbyl/heres_my_latest_video_main_topic_consuming/gomkg3y/,0


In [80]:
diabetes_test.to_csv('diabetes_classified_svm.csv', index = False)